In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# AI Explanations: Local Experience

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/ai-platform-samples/tree/master/notebooks/samples/explanations/tf2/ai-explanations-local-experience.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/ai-platform-samples/tree/master/notebooks/samples/explanations/tf2/ai-explanations-local-experience.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

This tutorial shows how to generate explanations for a model deployed locally using the AI Platform Explanations service. The AI Platform Explanations service is used to get feature attributions on your deployed model --explanations on how the features that contributed to the prediction.

The local experience feature is only accessible from an AI Platform notebook with Tensorflow 2.3 framework. It will not run in Colab or your laptop, or on n AIP instance with an earlier version of Tensorflow.

### Dataset

The dataset used for this tutorial is the [flowers dataset](https://www.tensorflow.org/datasets/catalog/tf_flowers) from [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview).

### Objective

The goal of this tutorial is to undetstand how to generate explanations for a model deployed locally using the AI Explanations service to obtain explanations for prediction from an image (flower) classification model. For image models, AI Explanations returns an image with the pixels highlighted that signaled your model's prediction the most.

This tutorial focuses more on deploying the model to AI Platform with Explanations than on the design of the model itself. 

### Costs

This tutorial uses billable components of Google Cloud Platform (GCP):

* AI Platform for:
  * Prediction
  * Explanation: AI Explanations comes at no extra charge to prediction prices. However, explanation requests take longer to process than normal predictions, so heavy usage of Explanations along with auto-scaling may result in more nodes being started and thus more charges
* Cloud Storage for:
  * Storing model files for deploying to Cloud AI Platform

Learn about [AI Platform
pricing](https://cloud.google.com/ml-engine/docs/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Before you begin

**Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select Runtime --> Change runtime type**

This tutorial assumes you are running the notebook either in **Colab** or **Cloud AI Platform Notebooks**.

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project.](https://console.cloud.google.com/cloud-resource-manager)

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the AI Platform Training & Prediction and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Project ID

**If you don't know your project ID.**

You might able to get your project ID using `gcloud` command, by executing the second code cell below.

In [ ]:
PROJECT_ID = "python_docs_samples_tests"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "python_docs_samples_tests":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where Cloud
AI Platform services are
available](https://cloud.google.com/ml-engine/docs/tensorflow/regions). You can
not use a Multi-Regional Storage bucket for training with AI Platform.

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append onto the name of resources which will be created in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you submit a training job using the Cloud SDK, you upload a Python package
containing your training code to a Cloud Storage bucket. AI Platform runs
the code from this package. In this tutorial, AI Platform also saves the
trained model that results from your job in the same bucket. You can then
create an AI Platform model version based on this output in order to serve
online predictions.

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets. 

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "_xai_metadata_" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

### Import libraries

Import the libraries for this tutorial.

In [ ]:
import json
import os
import random
from base64 import b64encode

import googleapiclient
import numpy as np
import PIL
import tensorflow as tf
from matplotlib import pyplot as plt

# should be >= 2.1
print("Tensorflow version " + tf.__version__)
if tf.__version__ < "2.1":
    raise Exception("TF 2.1 or greater is required")

AUTO = tf.data.experimental.AUTOTUNE
print("AUTO", AUTO)

Import the Explanability SDK.

In [ ]:
import explainable_ai_sdk as xai

## Download and preprocess the data

This section shows how to download the flower images, use the `tf.data` API to create a data input pipeline, and split the data into training and validation sets.

In [ ]:
# GCS location of TFRecords for flowers dataset
GCS_PATTERN = "gs://flowers-public/tfrecords-jpeg-192x192-2/*.tfrec"

# Input size (Height, Width) for the Model
IMAGE_SIZE = [192, 192]
# Class labels for prediction
CLASSES = [
    "daisy",
    "dandelion",
    "roses",
    "sunflowers",
    "tulips",
]  # do not change, maps to the labels in the data (folder names)

# Batch size for training
BATCH_SIZE = 32
# Validation split for training
VALIDATION_SPLIT = 0.2


# Split data files between training and validation
filenames = tf.io.gfile.glob(GCS_PATTERN)
random.shuffle(filenames)
split = int(len(filenames) * VALIDATION_SPLIT)
training_filenames = filenames[split:]
validation_filenames = filenames[:split]
print(
    "Pattern matches {} data files. Splitting dataset into {} training files and {} validation files".format(
        len(filenames), len(training_filenames), len(validation_filenames)
    )
)
validation_steps = int(3670 // len(filenames) * len(validation_filenames)) // BATCH_SIZE
steps_per_epoch = int(3670 // len(filenames) * len(training_filenames)) // BATCH_SIZE
print(
    "With a batch size of {}, there will be {} batches per training epoch and {} batch(es) per validation run.".format(
        BATCH_SIZE, steps_per_epoch, validation_steps
    )
)

The following cell contains some image visualization utility functions. This code isn't essential to training or deploying the model. 

If you're running this from Colab the cell is hidden. You can look at the code by right clicking on the cell --> "Form" --> "Show form" if you'd like to see it.

In [ ]:
# @title display utilities [RUN ME]


def dataset_to_numpy_util(dataset, N):
    dataset = dataset.batch(N)

    if tf.executing_eagerly():
        #  In eager mode, iterate in the Dataset directly.
        for images, labels in dataset:
            numpy_images = images.numpy()
            numpy_labels = labels.numpy()
            break
    else:
        # In non-eager mode, must get the TF note that
        # yields the nextitem and run it in a tf.Session.
        get_next_item = dataset.make_one_shot_iterator().get_next()
        with tf.Session() as ses:
            numpy_images, numpy_labels = ses.run(get_next_item)

    return numpy_images, numpy_labels


def title_from_label_and_target(label, correct_label):
    label = np.argmax(label, axis=-1)  # one-hot to class number
    correct_label = np.argmax(correct_label, axis=-1)  # one-hot to class number
    correct = label == correct_label
    return (
        "{} [{}{}{}]".format(
            CLASSES[label],
            str(correct),
            ", shoud be " if not correct else "",
            CLASSES[correct_label] if not correct else "",
        ),
        correct,
    )


def display_one_flower(image, title, subplot, red=False):
    plt.subplot(subplot)
    plt.axis("off")
    plt.imshow(image)
    plt.title(title, fontsize=16, color="red" if red else "black")
    return subplot + 1


def display_9_images_from_dataset(dataset):
    subplot = 331
    plt.figure(figsize=(13, 13))
    images, labels = dataset_to_numpy_util(dataset, 9)
    for i, image in enumerate(images):
        title = CLASSES[np.argmax(labels[i], axis=-1)]
        subplot = display_one_flower(image, title, subplot)
        if i >= 8:
            break

    plt.tight_layout()
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.show()


def display_9_images_with_predictions(images, predictions, labels):
    subplot = 331
    plt.figure(figsize=(13, 13))
    for i, image in enumerate(images):
        title, correct = title_from_label_and_target(predictions[i], labels[i])
        subplot = display_one_flower(image, title, subplot, not correct)
        if i >= 8:
            break

    plt.tight_layout()
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.show()


def display_training_curves(training, validation, title, subplot):
    if subplot % 10 == 1:  # set up the subplots on the first call
        plt.subplots(figsize=(10, 10), facecolor="#F0F0F0")
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor("#F8F8F8")
    ax.plot(training)
    ax.plot(validation)
    ax.set_title("model " + title)
    ax.set_ylabel(title)
    ax.set_xlabel("epoch")
    ax.legend(["train", "valid."])

### Read images and labels from TFRecords

In this dataset each image is stored as a`tf.Example` in a `TFRecords` file, which is a serialized format for storing and retrieving training data from a disk storage. To learn more about `tf.Example` and `TFRecords`, you can view the Tensorflow documentation [here](https://www.tensorflow.org/tutorials/load_data/tfrecord).

For our purpose, each image and corresponding label is encoded as a `tf.Example` as:

- image: JPG compressed image bytes
- class: the label (class) as a scalar (integer) value
- one_hot_class: the label (class) as a one-hot encoded vector

Our helper function `read_tfrecord` takes an individual `tf.Example`, deserializes using the specification defined in the variable `features`, extracts the image bytes and corresponding label, and JPG decodes (uncompresses) the image bytes.

Our helper function `load_datasets` creates a `tf.data.Dataset` from the corresponding `TFRecords` specified by the parameter `filenames`.

In [ ]:
def read_tfrecord(example):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),  # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means scalar
        "one_hot_class": tf.io.VarLenFeature(tf.float32),
    }
    example = tf.io.parse_single_example(example, features)
    image = tf.image.decode_jpeg(example["image"], channels=3)
    image = (
        tf.cast(image, tf.float32) / 255.0
    )  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMAGE_SIZE, 3])  # explicit size will be needed for TPU
    one_hot_class = tf.sparse.to_dense(example["one_hot_class"])
    one_hot_class = tf.reshape(one_hot_class, [5])
    return image, one_hot_class


def load_dataset(filenames):
    # Read data from TFRecords

    dataset = tf.data.Dataset.from_tensor_slices(filenames)
    dataset = dataset.interleave(
        tf.data.TFRecordDataset, cycle_length=16, num_parallel_calls=AUTO
    )  # faster
    dataset = dataset.map(read_tfrecord, num_parallel_calls=AUTO)
    return dataset

Use the visualization utility function provided earlier to preview flower images with their labels.

In [ ]:
display_9_images_from_dataset(load_dataset(training_filenames))

### Create training and validation datasets

Next, we will use our helper functions from the previous code cell to load the training and validation data into tf.data.Datasets, and set dataset properties for:

- cache(): Reads the dataset once from disk, which is then held in memory during training.
- repeat(): Enables multiple passes over the dataset for training more than a single epoch.
- shuffle(): The number of examples to random shuffle at a time.
- batch(): The batch size for training.
- prefetch(): With AUTO setting, the dataset iterator will auto determine when/how many examples to prefetch in parallel while feeding examples for training.

Finally, we will prefetch a batch of examples from the validation dataset to use later for prediction and explainability --which we store as `some_flowers` and `some_labels`.

In [ ]:
def get_batched_dataset(filenames):
    dataset = load_dataset(filenames)
    dataset = dataset.cache()  # This dataset fits in RAM
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(
        AUTO
    )  # prefetch next batch while training (autotune prefetch buffer size)
    # For proper ordering of map/batch/repeat/prefetch, see Dataset performance guide: https://www.tensorflow.org/guide/performance/datasets
    return dataset


def get_training_dataset():
    return get_batched_dataset(training_filenames)


def get_validation_dataset():
    return get_batched_dataset(validation_filenames)


some_flowers, some_labels = dataset_to_numpy_util(
    load_dataset(validation_filenames), BATCH_SIZE
)

## Build, train, and evaluate the model

This section shows how to build, train, evaluate, and get local predictions from a model by using the TF.Keras [Sequential API](https://www.tensorflow.org/guide/keras/sequential_model).

Note that when we loaded and deserialized the `tf.Example` records, we extracted the one-hot-encoded version of the label. So when we `compile()` the model, we use `categorical_crossentropy` for the loss function.

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (BatchNormalization, Conv2D, Dense,
                                     GlobalAveragePooling2D, MaxPooling2D)
from tensorflow.keras.optimizers import Adam

model = Sequential(
    [
        # Stem
        Conv2D(
            kernel_size=3,
            filters=16,
            padding="same",
            activation="relu",
            input_shape=[*IMAGE_SIZE, 3],
        ),
        BatchNormalization(),
        Conv2D(kernel_size=3, filters=32, padding="same", activation="relu"),
        BatchNormalization(),
        MaxPooling2D(pool_size=2),
        # Conv Group
        Conv2D(kernel_size=3, filters=64, padding="same", activation="relu"),
        BatchNormalization(),
        MaxPooling2D(pool_size=2),
        Conv2D(kernel_size=3, filters=96, padding="same", activation="relu"),
        BatchNormalization(),
        MaxPooling2D(pool_size=2),
        # Conv Group
        Conv2D(kernel_size=3, filters=128, padding="same", activation="relu"),
        BatchNormalization(),
        MaxPooling2D(pool_size=2),
        Conv2D(kernel_size=3, filters=128, padding="same", activation="relu"),
        BatchNormalization(),
        # 1x1 Reduction
        Conv2D(kernel_size=1, filters=32, padding="same", activation="relu"),
        BatchNormalization(),
        # Classifier
        GlobalAveragePooling2D(),
        Dense(5, activation="softmax"),
    ]
)

model.compile(
    optimizer=Adam(lr=0.005, decay=0.98),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
model.summary()

### Train the model

Train this on a GPU if you have access (in Colab, from the menu select **Runtime** --> **Change runtime type**). On a CPU, it'll take ~30 minutes to run training. On a GPU, it takes ~5 minutes.

After the model is trained, we will print out the validation accuracy per epoch from the model's training `history`.

In [ ]:
EPOCHS = 20  # Train for 60 epochs for higher accuracy, 20 should get you ~75%

history = model.fit(
    get_training_dataset(),
    steps_per_epoch=steps_per_epoch,
    epochs=EPOCHS,
    validation_data=get_validation_dataset(),
    validation_steps=validation_steps,
)

print(history.history["val_accuracy"])

### Visualize local predictions

Get predictions on your local model and visualize the images with their predicted labels, using the visualization utility function provided earlier.

In [ ]:
# Randomize the input so that you can execute multiple times to change results
permutation = np.random.permutation(BATCH_SIZE)
some_flowers, some_labels = (some_flowers[permutation], some_labels[permutation])

# Get predictions from our sample batch
predictions = model.predict(some_flowers, batch_size=16)
print("Predictions", np.array(CLASSES)[np.argmax(predictions, axis=-1)].tolist(), "\n")

# Get evaluations (true labels) from our sample batch
evaluations = model.evaluate(some_flowers, some_labels, batch_size=16)
print("Evaluation [val_loss, val_acc]", evaluations)

In [ ]:
display_9_images_with_predictions(some_flowers, predictions, some_labels)

## Export the model as a TF 2.x SavedModel

When using TensorFlow 2.x, you export the model as a `SavedModel` and load it into Cloud Storage. During export, you need to define a serving function to convert data to the format your model expects. If you send encoded data to AI Platform, your serving function ensures that the data is decoded on the model server before it is passed as input to your model.

### Serving function for image data

Sending base 64 encoded image data to AI Platform is more space efficient. Since this deployed model expects input data as raw bytes, you need to ensure that the b64 encoded data gets converted back to raw bytes before it is passed as input to the deployed model.

To resolve this, define a serving function (`serving_fn`) and attach it to the model as a preprocessing step. Add a `@tf.function` decorator so the serving function is part of the model's graph (instead of upstream on a CPU).

When you send a prediction or explanation request, the request goes to the serving function (`serving_fn`), which preprocesses the b64 bytes into raw numpy bytes (`preprocess_fn`). At this point, the data can be passed to the model (`m_call`).

In [ ]:
export_path = "gs://" + BUCKET_NAME + "/explanations/mymodel"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(192, 192))
    return resized


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    with tf.device("cpu:0"):
        decoded_images = tf.map_fn(_preprocess, bytes_inputs, dtype=tf.float32)
    return {
        "numpy_inputs": decoded_images
    }  # User needs to make sure the key matches model's input


m_call = tf.function(model.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 192, 192, 3], dtype=tf.float32, name="numpy_inputs")]
)


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


tf.saved_model.save(
    model,
    export_path,
    signatures={
        "serving_default": serving_fn,
        "xai_preprocess": preprocess_fn,  # Required for XAI
        "xai_model": m_call,  # Required for XAI
    },
)

## Get signatures

You can get the signatures of your model's input and output layers by reloading the model into memory, and querying it for the signatures corresponding to each layer.

When you subsequently generate metadata for explanations, you will need the signature of the input layer to override the metadata builder defaulting to using the serving function's input signature.

In [ ]:
loaded = tf.saved_model.load(export_path)

input_name = list(loaded.signatures["xai_model"].structured_input_signature[1].keys())[
    0
]
print(input_name)

output_name = list(loaded.signatures["xai_model"].structured_outputs.keys())[0]
print(output_name)

preprocess_name = list(
    loaded.signatures["xai_preprocess"].structured_input_signature[1].keys()
)[0]
print(preprocess_name)

## Generate explanation metadata

In order to deploy this model to AI Platform Explanations, we need to create an explanation_metadata.json file with information about our model inputs, outputs, and baseline. We can automatically generate and export this file by using Metadata builder from the AI Platform Explainability SDK.

### Input baseline

For image models, using [0,1] as your input baseline represents black and white images. In this case we're using np.random to generate the baseline because our training images contain a lot of black and white (i.e. daisy petals).

Let's start by first creating the input baseline.

In [ ]:
random_baseline = np.random.rand(192, 192, 3)

### Metadata builder

Next, you will instantiate an instance of SavedModelMetadatabuilder, with the parameters:

- model_path: path to the SavedModel format stored model. In this case, the path is the model stored in your Cloud Storage bucket, specified by `export_path`.
- signature_name: overrides looking at the default signature to find the input and output layers for explanations.

In [ ]:
from explainable_ai_sdk.metadata.tf.v2 import SavedModelMetadataBuilder

# By default models inputs and outputs are added to metadata.
builder = xai.metadata.tf.v2.SavedModelMetadataBuilder(
    export_path, signature_name="xai_model"
)

Next, we set some parameters for generating the metadata with the method `set_image_metadata`, with the parameters:

- input_name: the signature of the input layer to override using the serving function.
- input_baseline: the input baseline.
- visualization: how the explanation data should be visualized.

In [ ]:
# We add actual float image to explain as b64 string can't be explained.
builder.set_image_metadata(
    input_name,
    input_baselines=[random_baseline.tolist()],
    visualization={"type": "Pixels", "overlay_type": "grayscale"},
)

### Upload the metadata

The last step in generating the metadata for explanations, is to upload the generated metadata json to where the stored model using the method `save_model_with_metadata`.

In [ ]:
# Save the model with metadata to our GCS export path.
export_path = builder.save_model_with_metadata(export_path)

## Local experience - predictions and explanations

Now that you have a AI Platform model with the corresponding predictions, we can export the model to a local instance of an AI Plarform notebook and make local predictions with the corresponding explanations.

The first step is to export the model locally. We will use the AI Platform Explanability SDK method `load_model_from_local_path()`, with the parameters:

- `model_path`: The GCS location of the AI Platform model.
- `config`: The type of XAI explanations to instrument the local model for.

In [ ]:
lm = xai.load_model_from_local_path(export_path, xai.XraiConfig())

### Get and prepare test images

To prepare the test images:

- Download a small sample of images from the flowers dataset -- just enough for a multiple instance prediction (3 in total).
- Resize the images to match the input shape (192, 192) of the model.
- Save the resized images back to your bucket.

In [ ]:
# Download test flowers from public bucket
! mkdir flowers
! gsutil -m cp gs://flowers_model/test_flowers/* ./flowers

# Resize the images to what your model is expecting (192,192)
test_filenames = []

for i in os.listdir("flowers"):
    img_path = "flowers/" + i
    with PIL.Image.open(img_path) as ex_img:
        resize_img = ex_img.resize([192, 192])
        resize_img.save(img_path)
        test_filenames.append(img_path)

### Format your explanation request

Prepare a batch of instances in JSONL format.

In [ ]:
# Prepare your prediction JSON to send to your Cloud model
instances = []

for image_path in test_filenames:
    img_bytes = tf.io.read_file(image_path)
    b64str = b64encode(img_bytes.numpy()).decode("utf-8")
    instances.append({preprocess_name: {"b64": b64str}})

### Make prediction with explanation

Now that you have a local version of your model on your AI Platform notebook, we can make a prediction from the three test images and get corresponding explanations using the `explain()` method for your model.

In [ ]:
explanations = lm.explain(instances)

### Visualize the predictions and explanations

Let's now visualize the explanations for the three test images from the response object to the `explain()` call. This response object will return a list, where each element is the explanation data for the corresponding test image. You will use the method `visualize_attributions()` to visualize the explanation.

In [ ]:
explanations[0].visualize_attributions()
explanations[1].visualize_attributions()
explanations[2].visualize_attributions()

## Deploy model to AI Explanations

This section shows how to use `gcloud` to deploy the model to AI Explanations, using two different explanation methods for image models. After the model is deployed, we will export a local copy of the deployed model to your AI Platform notebook instance and repeat making a prediction and corresponding explanations.

### Create the model

In [ ]:
import datetime

MODEL = "flowers" + TIMESTAMP
print(MODEL)

In [ ]:
# Create the model if it doesn't exist yet (you only need to run this once)
! gcloud ai-platform models create $MODEL --enable-logging --regions=us-central1

### Create explainable model versions

For image models, we offer two choices for explanation methods: 
* Integrated Gradients (IG)
* XRAI 

You can find more info on each method in the [documentation](TODO). In this tutorial, you will deploy for the XRAI explanation method.

Creating the version will take ~5-10 minutes. Note that your first deploy may take longer.

#### Deploy an XRAI model

In [ ]:
XRAI_VERSION = "xrai"

# Create the XRAI version with gcloud
! gcloud beta ai-platform versions create $XRAI_VERSION \
--model $MODEL \
--origin $export_path \
--runtime-version 2.2 \
--framework TENSORFLOW \
--python-version 3.7 \
--machine-type n1-standard-4 \
--explanation-method xrai \
--num-integral-steps 25

In [ ]:
# Make sure the XRAI model deployed correctly. State should be `READY` in the following log
! gcloud ai-platform versions describe $XRAI_VERSION --model $MODEL

### Export deployed model with explanability to your local notebook instance

Finally, you will repeat the same explanability task, but this time the exported model will be the deployed model (vs. previously was the model resource). You export the deployed model using the `load_model_from_ai_platform()` method of the AI Explainability SDK.

In [ ]:
lm = xai.load_model_from_ai_platform(PROJECT_ID, MODEL, XRAI_VERSION)

### Make prediction with explanation

Now that you have a local version of your deployed model on your AI Platform notebook, we can repeat making a prediction from the three test images and get corresponding explanations using the `explain()` method for your model.

In [ ]:
explanations = lm.explain(instances)

### Visualize the predictions and explanations

Let's now visualize the explanations for the three test images from the response object to the `explain()` call. This response object will return a list, where each element is the explanation data for the corresponding test image. You will use the method `visualize_attributions()` to visualize the explanation.

In [ ]:
explanations[0].visualize_attributions()
explanations[1].visualize_attributions()
explanations[2].visualize_attributions()

## Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Alternatively, you can clean up individual resources by running the following
commands:

In [ ]:
# Delete model version resource
! gcloud ai-platform versions delete $IG_VERSION --quiet --model $MODEL
! gcloud ai-platform versions delete $XRAI_VERSION --quiet --model $MODEL

# Delete model resource
! gcloud ai-platform models delete $MODEL --quiet

# Delete Cloud Storage objects that were created
! gsutil -m rm -r gs://$BUCKET_NAME

If your Cloud Storage bucket doesn't contain any other objects and you would like to delete it, run `gsutil rm -r gs://$BUCKET_NAME`.

## What's next?

To learn more about AI Explanations, check out the resources here.

* [AI Explanations documentation](https://cloud.google.com/ai-platform/prediction/docs/ai-explanations/overview)
* [AI Explanations whitepaper](https://storage.googleapis.com/cloud-ai-whitepapers/AI%20Explainability%20Whitepaper.pdf)
* [Integrated gradients paper](https://arxiv.org/abs/1703.01365)
* [XRAI paper](https://arxiv.org/abs/1906.02825)